In [ ]:
# Generate Random CIK Numbers from SEC Website 

import pandas as pd
df = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
random_sample = df.sample(n=85)
company_list = random_sample.values.tolist()
cik_numbers = []
for i in company_list:
    cik_numbers.append(i[1])
print(cik_numbers)

In [ ]:
# Add 90 Days to 10-Q Filing Date

# import pandas, timedelta
import pandas as pd
from datetime import timedelta
   
# read csv data
df1 = pd.read_csv('10_Q_FILINGS.csv')
df1['date.filed'] = pd.to_datetime(df1['date.filed']) 
df1['90_DAYS'] = df1["date.filed"] + timedelta(days=90)
df2 = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
df2.columns = ['ticker', 'cik']
df2['ticker'] = df2['ticker'].str.upper()

   
df_merged = pd.merge(df1, 
                      df2, 
                      on ='cik', 
                      how ='inner')


quarterly_df = (df_merged[['cik','company.name', 'ticker', 'date.filed', '90_DAYS']])
#quarterly_df.to_csv('90_DAYS.csv')

In [ ]:
# Get Stock Price for Quarterly Date + 90 Days

# Import yfinance and matplotlib
import yfinance as yf  
import matplotlib.pyplot as plt

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['date.filed'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': 'date.filed'})
df_filed_merged = pd.merge(quarterly_df, 
                      df_ohlc, 
                      how ='inner',
                      on =['date.filed', 'ticker'] )
df_filed_merged = df_filed_merged.rename(columns={'Close': 'Filed_Price'})
print(df_filed_merged)

In [ ]:
# Import yfinance and matplotlib
import yfinance as yf  
import matplotlib.pyplot as plt

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['90_DAYS'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': '90_DAYS'})
df_quarterly_merged = pd.merge(df_filed_merged, 
                      df_ohlc, 
                      how ='inner',
                      on =['90_DAYS', 'ticker'] )
df_quarterly_merged = df_quarterly_merged.rename(columns={'Close': '90_Days_Close_Price', 'Filed_Price': 'Filed_Close_Price'})
df_quarterly_merged['Price_Change'] = df_quarterly_merged['90_Days_Close_Price'] - df_quarterly_merged['Filed_Close_Price']
print(df_quarterly_merged)
#df_quarterly_merged.to_csv('stock_prices_90_days.csv')

In [ ]:
#Calculate the Percent Change between the stock price at 10-Q filing date and after 90 days

# Import yfinance and matplotlib
import pandas as pd
import yfinance as yf  
import matplotlib.pyplot as plt
from datetime import timedelta

df1 = pd.read_csv('10_Q_FILINGS.csv')
df1['date.filed'] = pd.to_datetime(df1['date.filed']) 
df1['90_DAYS'] = df1["date.filed"] + timedelta(days=90)
df2 = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
df2.columns = ['ticker', 'cik']
df2['ticker'] = df2['ticker'].str.upper()

   
df_merged = pd.merge(df1, 
                      df2, 
                      on ='cik', 
                      how ='inner')


quarterly_df = (df_merged[['cik','company.name', 'ticker', 'date.filed', 'period.of.report', '90_DAYS']])

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['date.filed'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': 'date.filed'})
df_filed_merged = pd.merge(quarterly_df, 
                      df_ohlc, 
                      how ='inner',
                      on =['date.filed', 'ticker'] )
df_filed_merged = df_filed_merged.rename(columns={'Close': 'Filed_Price'})

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['90_DAYS'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': '90_DAYS'})
df_quarterly_merged = pd.merge(df_filed_merged, 
                      df_ohlc, 
                      how ='inner',
                      on =['90_DAYS', 'ticker'] )
df_quarterly_merged = df_quarterly_merged.rename(columns={'Close': '90_Days_Close_Price', 'Filed_Price': 'Filed_Close_Price'})
df_quarterly_merged['Price_Change'] = df_quarterly_merged['90_Days_Close_Price'] - df_quarterly_merged['Filed_Close_Price']
def percentage_change(col1,col2):
    return ((col2 - col1) / col1)
df_quarterly_merged['Pct_Change_Price'] = percentage_change(df_quarterly_merged['Filed_Close_Price'],df_quarterly_merged['90_Days_Close_Price'])
print(df_quarterly_merged)
df_quarterly_merged.to_csv('stock_prices_90_days_pct_changev2.csv')

In [ ]:
#Get Free Cash Flow for Sample Companies from MacroTrends financial charts website
#Get this dataframe into df_quarterly merged
#Create YOY function
"""
Created on Tue Apr  5 20:36:06 2022

@author: conno
"""
from enum import unique
from numpy import inner
import requests
import time
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta


def getAllFCF(df):
    tickers = df['ticker'].unique()
    allfcf = pd.DataFrame()
    for i in tickers:
        url = df.loc[df['ticker'] == i, 'url']
        fcf = getFCF(i, url.iloc[0])
        fcf['Ticker'] = i
        allfcf = allfcf.append(fcf, ignore_index=True)
    return allfcf


def getFCF(ticker, url):
    page = requests.get("https://www.macrotrends.net/stocks/charts/" + ticker + "/" + url + "/free-cash-flow")
    if page.status_code != 200:
        return pd.DataFrame()
    else:    
    
        soup = BeautifulSoup(page.content, 'html.parser')
        tables = soup.find_all(class_="historical_data_table")
        text = tables[1]
        

        table_rows = text.find_all('tr')

        rows = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [i.text for i in td]
            rows.append(row)

        df = pd.DataFrame(rows, columns =['Date', 'Value'])
        return df

def convertCompanyNames(names, df):
    lst = []
    for i in names:
        name = i.lower()
        name = name.replace(" ","-")
        lst.append([i,name])
        
    df2 = pd.DataFrame(lst, columns=["company.name", "url"])
    return pd.merge(df,df2,on="company.name", how="inner")



df1 = pd.read_csv('10_Q_FILINGS.csv')
df1['date.filed'] = pd.to_datetime(df1['date.filed']) 
df1['90_DAYS'] = df1["date.filed"] + timedelta(days=90)
df2 = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
df2.columns = ['ticker', 'cik']
df2['ticker'] = df2['ticker'].str.upper()

    
df_merged = pd.merge(df1, df2, on ='cik', how ='inner')


companies = df_merged['company.name'].unique()

df3 = convertCompanyNames(companies, df_merged)

    
df3['url'] = df3['url'].str.replace('-$', '')
df3['url'] = df3['url'].str.replace('--', '-')
    
allfcf = getAllFCF(df3)
file_output = allfcf.to_csv('output_fcf.csv')


In [ ]:
#Categorization of Quarters for FCF dataframe
from datetime import datetime
from datetime import date as d

def addQuarters(date):
    
    if date is not None:
        date = date.replace('-', '/')

        quarters = {'Q1':[[1,1],[3,31]],'Q2':[[4,1],[6,30]],'Q3':[[7,1],[9,30]],'Q4':[[10,1],[12,31]]}


        for i in range(1,5):
            date_time_obj = datetime.strptime(date, '%Y/%m/%d').date()

            d1 = d(date_time_obj.year, quarters['Q'+str(i)][0][0], quarters['Q'+str(i)][0][1])
            d2 = d(date_time_obj.year, quarters['Q'+str(i)][1][0], quarters['Q'+str(i)][1][1])

            if d1 <= date_time_obj and d2 >= date_time_obj:
                return('Q' + str(i)), str(date_time_obj.year)
    return "NA","NA"

    

In [ ]:

allfcf['Quarter'] = allfcf['Date'].apply(addQuarters)


                                

In [146]:
#Find row where quarter tuple and ticker are the same
import re

def addPerctageIncYOY(row,allfcf):
    if row['Quarter'][0] != "NA":
        prevYear = (row['Quarter'][0], str(int(row['Quarter'][1]) - 1))
        df = allfcf.loc[allfcf['Ticker'] == row['Ticker']]
        df = df.loc[df['Quarter'] == prevYear]
        if len(df) == 1:
            
            r = row['Value']
            r = re.sub(r'[^\x00-\x7F]+','-', r)
            r = r.replace(',','')
            
            
        
            b = df['Value'].iloc[0]
            b = re.sub(r'[^\x00-\x7F]+','-', b)
            b = b.replace(',','')
           
            if r == '' or b == '':
                return "NA"
            if b == "0.00" or b == "-0.00" :
                return "NA"
            return str(((float(r) - float(b))/float(b))*100)
        else:
            return "NA"
    
    
    

In [ ]:
allfcf['PctIncYOY'] = allfcf.apply(addPerctageIncYOY,allfcf=allfcf,axis=1)


In [179]:
#test = allfcf.loc[allfcf['Ticker'] == 'TTEC']

#len(test.loc[test['Quarter'] == ("Q1","2000")])
def changeDashes(x):
    if x is not None:
        x = x.replace('-','/')
        return x
    return "NA"

def changeOrder(x):
    if x != "NA":
        date_time_obj = datetime.strptime(x, '%Y/%m/%d').date()
        
        return date_time_obj.strftime('%#m/%d/%y')
    return "NA"

allfcf['period.of.report'] = allfcf['Date'].apply(changeOrder)
allfcf.head()
                              

,Date,Value,ticker,Quarter,PctIncYOY,DateAdj,period.of.report
0,NA,None,TTEC,"(NA, NA)",None,NA,NA
1,2021/09/30,134.33,TTEC,"(Q3, 2021)",-3.3388501115348537,09/30/21,9/30/21
2,2021/06/30,109.28,TTEC,"(Q2, 2021)",48.92341237394386,06/30/21,6/30/21
3,2021/03/31,58.25,TTEC,"(Q1, 2021)",28.388803173903465,03/31/21,3/31/21
4,2020/12/31,212.17,TTEC,"(Q4, 2020)",19.465090090090087,12/31/20,12/31/20


In [165]:
names = allfcf.columns.tolist()
names[names.index('Ticker')] = 'ticker'
allfcf.columns = names
allfcf.head()

,Date,Value,ticker,Quarter,PctIncYOY,DateAdj,period.of.report
0,NA,None,TTEC,"(NA, NA)",None,NA,NA
1,2021/09/30,134.33,TTEC,"(Q3, 2021)",-3.3388501115348537,09/30/21,09/30/21
2,2021/06/30,109.28,TTEC,"(Q2, 2021)",48.92341237394386,06/30/21,06/30/21
3,2021/03/31,58.25,TTEC,"(Q1, 2021)",28.388803173903465,03/31/21,03/31/21
4,2020/12/31,212.17,TTEC,"(Q4, 2020)",19.465090090090087,12/31/20,12/31/20


In [216]:
df_linReg = pd.merge(df_quarterly_merged, 
                      allfcf, 
                      how ='inner',
                      on =['ticker','period.of.report'])


df_model = df_linReg[['PctIncYOY','Price_Change']]
df_model


,PctIncYOY,Price_Change
0,-3012.4999999999995,8.399998
1,46.71846815595949,-5.000000
2,-4.816404387219829,-6.549999
3,-8.405464594848267,-7.529999
4,27.000189143181387,8.230000
...,...,...
409,1703.3665244191561,9.669998
410,370.8368084514874,-11.160004
411,-33.85260635110844,0.030001
412,-184.0943002088929,3.960001


In [219]:
def changeFloat(x):
    if x != "NA":
        return float(x)
    return "NA"

df_model['Price_Change'] = df_model['Price_Change'].apply(changeFloat)
type(df_model['Price_Change'].iloc[0])

C:\Users\conno\AppData\Local\Temp/ipykernel_15624/3751229290.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Price_Change'] = df_model['Price_Change'].apply(changeFloat)


numpy.float64

In [225]:
import patsy
patsy.__version__

'0.5.2'

In [227]:
df_model.to_csv('model1.csv')

In [224]:
#Calculate Linear Regression
import statsmodels.formula.api as smf

# Initialise and fit linear regression model using `statsmodels`
model = smf.ols('% Incr YOY ~ Price Incr', data=df_model)
model = model.fit()


AssertionError: 

In [226]:
import patsy
patsy.__version__

'0.5.2'

In [177]:
price_pred = model.predict()

# Plot regression against actual data
plt.figure(figsize=(12, 6))
plt.plot(df_model['PctIncYOY'], df_model['Price_Change'], 'o')           # scatter plot showing actual data
plt.plot(df_model['PctIncYOY'], price_pred, 'r', linewidth=2)   # regression line
plt.xlabel('Percentage Increase YOY')
plt.ylabel('Price Increase')
plt.title('Percentage Increase YOY vs Price Increase')

plt.show()



,cik,company.name,ticker,date.filed,period.of.report,90_DAYS,Filed_Close_Price,90_Days_Close_Price,Price_Change,Pct_Change_Price
0,1013880,TELETECH HOLDINGS INC,TTEC,2017-05-09,3/31/17,2017-08-07,32.450001,40.849998,8.399998,0.258860
1,1013880,TELETECH HOLDINGS INC,TTEC,2017-11-08,9/30/17,2018-02-06,41.200001,36.200001,-5.000000,-0.121359
2,1013880,TTEC Holdings Inc,TTEC,2018-05-10,3/31/18,2018-08-08,32.799999,26.250000,-6.549999,-0.199695
3,1013880,TTEC Holdings Inc,TTEC,2018-08-07,6/30/18,2018-11-05,32.799999,25.270000,-7.529999,-0.229573
4,1013880,TTEC Holdings Inc,TTEC,2018-11-07,9/30/18,2019-02-05,25.430000,33.660000,8.230000,0.323633
5,1013880,TTEC Holdings Inc,TTEC,2019-05-07,3/31/19,2019-08-05,35.959999,43.750000,7.790001,0.216630
6,1013880,TTEC Holdings Inc,TTEC,2019-08-07,6/30/19,2019-11-05,44.919998,47.439999,2.520000,0.056100
7,1013880,TTEC Holdings Inc,TTEC,2019-11-05,9/30/19,2020-02-03,47.439999,40.730000,-6.709999,-0.141442
8,1013880,TTEC Holdings Inc,TTEC,2020-08-05,6/30/20,2020-11-03,52.459999,58.509998,6.049999,0.115326
9,1013880,TTEC Holdings Inc,TTEC,2020-11-04,9/30/20,2021-02-02,61.299999,81.070000,19.770000,0.322512


In [ ]:
#Calculate Free Cash Flow Percent Change from Previous Quarter

import pandas as pd
df_pct_change = pd.read_csv('output_fcf.csv', index_col=0)
df_pct_change = df_pct_change.dropna(how='any',axis=0) 
df_pct_change['Value'] = pd.to_numeric(df_pct_change['Value'],errors='coerce')
df_pct_change['pct'] = df_pct_change.sort_values('Date', axis=0, ascending=True).groupby(['Ticker']).Value.pct_change()
df_pct_change.to_csv('fcf_pct_change.csv')
print(df_pct_change)

In [ ]:
#Merge Percent Change of Free Cash Flow with Stock Price Change Dataframe


import pandas as pd
df_stock_prices = pd.read_csv('stock_prices_90_days_pct_changev2.csv')
df_fcf = pd.read_csv('fcf_pct_change.csv', index_col=0)
df_stock_prices = df_stock_prices.rename(columns = {'period.of.report': 'Date', 'ticker': 'Ticker'})
df_fcf_merged = pd.merge(df_fcf, 
                      df_stock_prices, 
                      how ='inner',
                      on =['Date','Ticker'] )

df_fcf_merged.drop(['Unnamed: 0'], axis=1)
df_fcf_merged = df_fcf_merged.rename(columns = {'Date': 'Period', 'pct': 'FCF_PCT_CHANGE', 'Value': 'FCF'})
print(df_fcf_merged)
df_fcf_merged.to_csv('stock_prices_90_days_FCF.csv')